# RSA Signature verification in U-Boot

<img src="secure_boot_in_uboot.png" alt="Verified boot in U-Boot" width="800"/>

U-Boot's new verified boot feature provides a mechanism for verifying images while still allowing them to be upgraded. Kernel image is loaded into memory by U-Boot and can be verified in U-Boot.

U-Boot provided utility functions to verify the signature. Currently U-Boot supports RSA signature verification encrypted on SHA1 / SHA256 message digests.

U-Boot has the infrastructure to verify kernel image in FIT format. If the kernel image is in legacy format, a simple wrapper can be written to verify its signature.

# RSA sign and verify using OpenSSL

Signature creation and verification can be performed using OpenSSL. OpenSSL uses public and private key files to validate and generate the signature respectively. However in case of U-Boot OpenSSL is not available and hence the verification is done using few parameters generated from public key. Read RSA sign and verify using Openssl : Behind the scene for more details.

In [1]:
# Create a file containing all lower case alphabets
!echo abcdefghijklmnopqrstuvwxyz > myfile.txt

# Generate 512 bit Private key
!openssl genrsa -out myprivate.pem 512

# Separate the public part from the Private key file.
!openssl rsa -in myprivate.pem -pubout > mypublic.pem

# Sign the file using sha1 digest and PKCS1 padding scheme
!openssl dgst -sha1 -sign myprivate.pem -out sha1.sign myfile.txt

# Verify the signature of file
!openssl dgst -sha1 -verify mypublic.pem -signature sha1.sign myfile.txt

Generating RSA private key, 512 bit long modulus
.......+++++++++++++++++++++++++++
....+++++++++++++++++++++++++++
e is 65537 (0x10001)
writing RSA key
Verified OK


# View the public key details

Public key pem file is in base64 format and can be printed. [Read RSA public key : Behind the scene](https://medium.com/@bn121rajesh/understanding-rsa-public-key-70d900b1033c) for more details. Modulus and public exponent are encoded in to the public key.

In [12]:
# Get modulus and public exponent from public key
!openssl rsa -pubin -inform PEM -text -noout < mypublic.pem

Public-Key: (512 bit)
Modulus:
    00:ec:20:80:b1:08:94:3c:35:68:0f:78:4f:62:e4:
    a8:f3:18:ad:46:28:f5:80:4b:b5:3a:dd:3a:76:76:
    10:31:e7:cc:d1:5f:0a:3c:c7:93:61:c3:5b:a5:55:
    8c:67:2d:9f:ce:80:8f:29:38:bd:e5:8a:78:15:f7:
    71:8e:b2:a4:91
Exponent: 65537 (0x10001)


# U-Boot RSA public key parameters

Signature verification in U-Boot is performed using the below parameters. All these parameters are available or generated from RSA public key. Read [signature.txt](http://git.denx.de/?p=u-boot.git;a=blob;f=doc/uImage.FIT/signature.txt;h=a7657226794baefd61f33448bbcad074a473f4b9;hb=refs/heads/master) for more details

- **rsa,num-bits:** Number of key bits (e.g. 2048)

- **rsa,modulus:** Modulus (N) as a big-endian multi-word integer

- **rsa,exponent:** Public exponent (E) as a 64 bit unsigned integer

- **rsa,r-squared:** $(2^{NumBits})^2$ as a big-endian multi-word integer

- **rsa,n0-inverse:** $ \left(\frac{-1}{modulus[0]}\right) \% 2^{32}$

## rsa,num-bits

Number of key bits can be obtained directly from public key.

In [13]:
# Get the key length in bits
!openssl rsa -pubin -inform PEM -text -noout < mypublic.pem | head -1

Public-Key: (512 bit)


## rsa,modulus

Modulus (N) is obtained directly from public key, but must be converted into big endian words so that it can be used by U-Boot utility functions.

Below utility function converts integer into big endian words

In [14]:
def big_endian_words(val):
    """Convert integer into big endian words"""
    # Convert val to hex
    str_val = hex(val)[2:]

    # Split the string into bytes
    str_bytes = [str_val[i : i + 2] for i in range(0, len(str_val), 2)]

    # Group bytes into group os 4 bytes
    sets = [str_bytes[i : i + 4] for i in range(0, len(str_bytes), 4)]

    # Reverse the bytes in each group (BIG endian)
    list(map(list.reverse, sets))

    # Join bytes in each group (WORD)
    sets = list(map(''.join, sets))

    # Convert to upper case
    sets = list(map(str.upper, sets))

    # Append '0x' to the word 
    sets = list(map('0x'.__add__, sets))
    
    return sets

Convert modulus into big endian word array. These words can be hardcoded in a file in u-boot or can be hard coded in device tree. (If kernel image is in FIT format

In [15]:
# Get the modulus from public key
modulus = !openssl rsa -pubin -inform PEM -text -noout < mypublic.pem | grep -v "Exponent\|Public\|Modulus"
modulus

['    00:ec:20:80:b1:08:94:3c:35:68:0f:78:4f:62:e4:',
 '    a8:f3:18:ad:46:28:f5:80:4b:b5:3a:dd:3a:76:76:',
 '    10:31:e7:cc:d1:5f:0a:3c:c7:93:61:c3:5b:a5:55:',
 '    8c:67:2d:9f:ce:80:8f:29:38:bd:e5:8a:78:15:f7:',
 '    71:8e:b2:a4:91']

In [16]:
# Remove new lines, spaces and :
mod = ''.join(modulus).replace('\n', '').replace(' ', '').replace(':', '')

# Convert modulus to integer
mod_int = int(mod, 16)

# Print the modulus in words
print(big_endian_words(mod_int))

['0xB18020EC', '0x353C9408', '0x4F780F68', '0xF3A8E462', '0x2846AD18', '0xB54B80F5', '0x763ADD3A', '0xE7311076', '0x0A5FD1CC', '0x6193C73C', '0x55A55BC3', '0x9F2D678C', '0x298F80CE', '0x8AE5BD38', '0x71F71578', '0x91A4B28E']


## rsa,exponent

Public exponent (E) is obtained directly from public key. U-boot supports public exponent up to 64 bits.
However public exponent is usually a default value (65537).

In [17]:
# Get the public exponent
!openssl rsa -pubin -inform PEM -text -noout < mypublic.pem | tail -1

Exponent: 65537 (0x10001)


## rsa,r-squared

R-Squared value is obtained from num-bits as below.

$$
RSquared = (2^{NumBits})^2
$$

R-Squared value is also represented in big endian word format. So we use the same procedure as in modulus

In [18]:
# Get the num bits from public key
numbits = !openssl rsa -pubin -inform PEM -text -noout < mypublic.pem | head -1 | grep -Eo "\d+"

# Convert num_bits to integer
num_bits = int(numbits[0])
num_bits

512

In [19]:
# Print the R-Squared in big endian words
print(big_endian_words((2 ** num_bits) ** 2 % mod_int))

['0x7A9286B3', '0xAFCFA072', '0xBA0F5A99', '0xFD494F91', '0x775BEEC7', '0xB8883A54', '0xF54A8DE5', '0x63150FF0', '0xE1D4189E', '0x36806001', '0x052C013D', '0x02019DEE', '0x2BD25E44', '0x60694872', '0x38AE42A6', '0x4E9F0330']


## rsa,n0-inverse

n0-inverse is the negative multiplicative inverse of modulus in $2^{32}$ modulo.

$$
n0inverse = \left(\frac{-1}{modulus[0]}\right) \% 2^{32}
$$

n0inverse is the solution to the equation:

$$
(modulus * n0inverse) \% 2^{32} = -1
$$

n0inverse is the integral solution for this diophantine equation:

$$
modulus * n0inverse = 2^{32} * k - 1
$$


Wolfram Alpha website provides solution to equations. Copy and paste the diophantine equation in [Wolfram Alpha website](https://www.wolframalpha.com) to get integral solutions.

In [20]:
# Diophantine equations can be solved using sympy package.
import sympy
from sympy.solvers.diophantine import diop_solve
inv, k = sympy.symbols('inv k')

# modulus[0] is the least significat word (in modulo 2**32)
eq = diop_solve((mod_int & 0xFFFFFFFF) * inv - (2 ** 32) * k + 1)
eq[0]

4294967296*t_0 + 745694095

In [21]:
# solutions for n0inverse
e = eq[0]
m = e.as_coefficients_dict()[list(e.free_symbols)[0]]
c = e.as_coefficients_dict()[1]
n0inverse = (m + c) % m
n0inverse

745694095

# References

1. [Verified Boot (From pacific simplicity)](https://pacificsimplicity.ca/blog/verified-boot-–-introduction-u-boot’s-secure-boot)
1. [Verified Boot (From U-Boot)](http://git.denx.de/?p=u-boot.git;a=blob;f=doc/uImage.FIT/verified-boot.txt;h=41c9fa9e09f97d047f980a31fcdbad5cd74d0db2;hb=refs/heads/master)
1. [Solving Diophantine (From Sympy)](https://docs.sympy.org/latest/modules/solvers/diophantine.html)
1. [Solving Diophantine (From WolFramAlpha)](https://www.wolframalpha.com)